In [1]:
import pandas as pd 
import matplotlib.pyplot as plt 
import numpy as np 

In [2]:
# Create a set of random latitude and longitude combinations 
lats = np.random.uniform(-90, 90, size = 1500)
lngs = np.random.uniform(-180, 180, size = 1500)
lat_lngs = zip(lats, lngs)
lat_lngs

In [3]:
from citipy import citipy

In [4]:
# Add the latitudes and longitudes to a list 
coordinates = list(lat_lngs)

In [5]:
# Create a list for holding the cities 
cities = []

# Identify the nearest city for each latitude and longitude combination
for coordinate in coordinates: 
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name 

    # If the city is unique, then add it to the cities list
    if city not in cities: 
        cities.append(city)

# Print the city count to confirm sufficient count 
len(cities)

627